In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_json('/content/script-bag-of-words.json')

In [3]:
df.head()

,episodeAlt,seasonNum,episodeNum,episodeTitle,text
0,S1E1,1,1,Winter Is Coming,"[{'text': 'Easy, boy.', 'name': 'Will'}, {'tex..."
1,S1E2,1,2,The Kingsroad,"[{'text': 'And You child... drink, eat. need t..."
2,S1E3,1,3,Lord Snow,[{'text': 'Council. Grand Lord Maester Pycelle...
3,S1E4,1,4,"Cripples, Bastards, and Broken Things",[{'text': 'Lord's The again. been dreaming lit...
4,S1E5,1,5,The Wolf and the Lion,[{'text': 'Does Hugh Ser any capital? family h...


In [4]:
df.tail()

,episodeAlt,seasonNum,episodeNum,episodeTitle,text
68,S8E2,8,2,A Knight of the Seven Kingdoms,"[{'name': 'Daenerys Targaryen', 'text': 'About..."
69,S8E3,8,3,The Long Night,"[{'name': 'Northman #1', 'text': 'Oi!'}, {'nam..."
70,S8E4,8,4,The Last of the Starks,"[{'name': 'Jon Snow', 'text': 'And Everyone It..."
71,S8E5,8,5,The Bells,"[{'name': 'Lord Varys', 'text': 'And? Come Not..."
72,S8E6,8,6,The Iron Throne,"[{'name': 'Tyrion Lannister', 'text': 'I'll fi..."


In [5]:
df.shape

(73, 5)

In [6]:
df.columns

Index(['episodeAlt', 'seasonNum', 'episodeNum', 'episodeTitle', 'text'], dtype='object')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   episodeAlt    73 non-null     object
 1   seasonNum     73 non-null     int64 
 2   episodeNum    73 non-null     int64 
 3   episodeTitle  73 non-null     object
 4   text          73 non-null     object
dtypes: int64(2), object(3)
memory usage: 3.0+ KB


In [8]:
df.isnull().sum()

episodeAlt      0
seasonNum       0
episodeNum      0
episodeTitle    0
text            0
dtype: int64

In [9]:
df.iloc[-1]['text']

[{'name': 'Tyrion Lannister', 'text': "I'll find later. you"},
 {'name': 'Jon Snow', 'text': "It's Let me men not safe. send some with you."},
 {'name': 'Tyrion Lannister', 'text': "I'm alone. going"},
 {'name': 'Grey Worm',
  'text': 'Daenerys I In Queen, Targaryen, die. name of one sentence the the to true you'},
 {'name': 'Jon Snow',
  'text': "Grey It's These Worm! are men over. prisoners."},
 {'name': 'Grey Worm',
  'text': "It Queen's are defeated. enemies is not over the until"},
 {'name': 'Davos Seaworth',
  'text': "How They're be? defeated do knees. more much on their them to want you"},
 {'name': 'Grey Worm', 'text': 'They are breathing.'},
 {'name': 'Davos Seaworth', 'text': 'Look We around friend. won. you,'},
 {'name': 'Grey Worm', 'text': "I commands, my not obey queen's yours."},
 {'name': 'Jon Snow', 'text': "And Queen's are commands? the what"},
 {'name': 'Grey Worm',
  'text': '"Kill Cersei Lannister." These They all are chose fight follow for free her. men. to who'}

In [10]:
dialouge = {}
for index, row in df.iterrows():
    for item in row['text']:
        if item['name'] in dialouge:
            dialouge[item['name']] = dialouge[item['name']] + item['text'] #appen
        else:
            dialouge[item['name']] = item['text'] + " " # character

In [11]:
len(dialouge)

817

In [12]:
new_df = pd.DataFrame()
new_df['character'] = dialouge.keys()
new_df['words'] = dialouge.values()

In [13]:
new_df.iloc[:,0:3].head()

,character,words
0,Will,"Easy, boy. I've I've Wildlings a a do ever in ..."
1,Waymar Royce,One They're What a and another before d'you ea...
2,Gared,Wall. We back head should the to Our They We W...
3,Jon Snow,Father's Go on. watching. And mother. yourBran...
4,Septa Mordane,"Fine Well always. as done. work, I Quite beaut..."


In [14]:
new_df['num_words'] = new_df['words'].apply(lambda x:len(x.split()))

In [15]:
new_df.iloc[:,0:3].head()

,character,words,num_words
0,Will,"Easy, boy. I've I've Wildlings a a do ever in ...",113
1,Waymar Royce,One They're What a and another before d'you ea...,94
2,Gared,Wall. We back head should the to Our They We W...,26
3,Jon Snow,Father's Go on. watching. And mother. yourBran...,11488
4,Septa Mordane,"Fine Well always. as done. work, I Quite beaut...",290


In [16]:
new_df = new_df.sort_values('num_words',ascending=False)

In [17]:
new_df = new_df.head(200)

In [18]:
new_df.shape

(200, 3)

In [19]:
new_df.head()

,character,words,num_words
17,Tyrion Lannister,It Mmh. Northern about girls. is say the they ...,25924
13,Cersei Lannister,And And Casterly One Rock. When about afraid. ...,14294
3,Jon Snow,Father's Go on. watching. And mother. yourBran...,11488
20,Daenerys Targaryen,We've a and anything. asked been for for guest...,11202
12,Jaime Lannister,"As I It's brother, duty feel it's much. my sho...",10823


In [20]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english')

In [21]:
embedding = cv.fit_transform(new_df['words']).toarray()

In [22]:
embedding.shape

(200, 16500)

In [23]:
embeddings = embedding.astype('float64')

In [24]:
from sklearn.manifold import TSNE

In [30]:
tsne = TSNE(n_components=2, perplexity=50.0, early_exaggeration=12.0, learning_rate='auto', n_iter=1000, n_iter_without_progress=300, min_grad_norm=1e-07,
            metric='euclidean', metric_params=None, init='pca', verbose=0, random_state=None, method='barnes_hut', angle=0.5, n_jobs=None)
z = tsne.fit_transform(embeddings)

In [31]:
z.shape

(200, 2)

In [32]:
new_df['x'] = z.T[0]
new_df['y'] = z.T[1]

In [33]:
new_df

,character,words,num_words,x,y
17,Tyrion Lannister,It Mmh. Northern about girls. is say the they ...,25924,4.273642,3.205743
13,Cersei Lannister,And And Casterly One Rock. When about afraid. ...,14294,4.128894,3.130829
3,Jon Snow,Father's Go on. watching. And mother. yourBran...,11488,4.123645,3.032118
20,Daenerys Targaryen,We've a and anything. asked been for for guest...,11202,3.886059,3.027468
12,Jaime Lannister,"As I It's brother, duty feel it's much. my sho...",10823,4.051220,3.040439
...,...,...,...,...,...
266,Matt Berninger,"""And ""That And And But Castamere. I I In Lord ...",96,-1.026688,0.092094
78,Mhaegen,"Lord? She She black don't hair. has him, his h...",95,-0.643813,0.176675
1,Waymar Royce,One They're What a and another before d'you ea...,94,-0.696313,-0.211663
156,Captain's Daughter,Is as it lord? my remember you Everything I I ...,93,-0.843336,-0.041039


In [36]:
import plotly.express as px
fig = px.scatter(new_df.head(50), x="x", y="y", color="character")
fig.show()

In [35]:
import pickle

# Dump the data into the .pkl file
with open('data_up.pkl', 'wb') as file:
    pickle.dump(new_df, file)

In [ ]:
pd.__version__

'1.5.3'